In [5]:
import cv2
import mediapipe as mp

# Inicializar MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# Cargar la imagen del sombrero y convertirla a RGBA
hat = cv2.imread('hat-crop.png', cv2.IMREAD_UNCHANGED)
hat_h, hat_w, _ = hat.shape  # Obtener dimensiones del sombrero

# Cargar la imagen del collar y convertirla a RGBA
ruff = cv2.imread('ruff-crop.png', cv2.IMREAD_UNCHANGED)
ruff_h, ruff_w, _ = hat.shape  # Obtener dimensiones del collar

# Variables de estado anteriores para suavizado
prev_x, prev_y, prev_w, prev_h = None, None, None, None
alpha = 0.2  # Factor de suavizado; entre más cercano a 1, menos suavizado

# Inicializar la cámara o cargar un video
cap = cv2.VideoCapture(0)  # Usa '0' para la cámara web

# Función para dibujar un rectángulo con esquinas redondeadas
def draw_rounded_rectangle(image, top_left, bottom_right, color, corner_radius, thickness=-1):
    x1, y1 = top_left
    x2, y2 = bottom_right

    # Dibujar círculos en las esquinas
    cv2.circle(image, (x1 + corner_radius, y1 + corner_radius), corner_radius, color, thickness)
    cv2.circle(image, (x2 - corner_radius, y1 + corner_radius), corner_radius, color, thickness)
    cv2.circle(image, (x1 + corner_radius, y2 - corner_radius), corner_radius, color, thickness)
    cv2.circle(image, (x2 - corner_radius, y2 - corner_radius), corner_radius, color, thickness)

    # Dibujar líneas entre los círculos
    cv2.rectangle(image, (x1 + corner_radius, y1), (x2 - corner_radius, y2), color, thickness)
    cv2.rectangle(image, (x1, y1 + corner_radius), (x2, y2 - corner_radius), color, thickness)



with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("No se puede acceder a la cámara.")
            break

       

        # Convertir la imagen a RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Procesar la imagen para detectar caras
        results = face_detection.process(image_rgb)

        # Convertir de nuevo a BGR para OpenCV
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        background_layer = image_bgr.copy() 
        
        # Dibujar los resultados de la detección en la imagen
        if results.detections:
            for detection in results.detections:
                # Dibuja el cuadro de la cara detectada
                #mp_drawing.draw_detection(image_bgr, detection)

                # Extraer el bounding box de la detección
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = image_bgr.shape  # Dimensiones de la imagen

                # Convertir coordenadas relativas a píxeles absolutos
                x = int(bboxC.xmin * iw)
                y = int(bboxC.ymin * ih)
                w = int(bboxC.width * iw)
                h = int(bboxC.height * ih)

                # Suavizado exponencial
                if prev_x is not None:
                    x = int(alpha * x + (1 - alpha) * prev_x)
                    y = int(alpha * y + (1 - alpha) * prev_y)
                    w = int(alpha * w + (1 - alpha) * prev_w)
                    h = int(alpha * h + (1 - alpha) * prev_h)
                
                # Actualizar las variables anteriores
                prev_x, prev_y, prev_w, prev_h = x, y, w, h

                # Copia temporal de la región de la cara en la bounding box
                face_region = image_bgr[y:y+h, x:x+w].copy()

                 # Calcular el rectángulo con proporción de carta de póker (5:7)
                card_w = int(w * 1.4)
                card_h = int(card_w * 7 / 5)
                card_x = x - int((card_w - w) / 2)
                card_y = y - int(h * 0.3)

                # Limitar los bordes del rectángulo dentro de la imagen
                card_x = max(0, card_x)
                card_y = max(0, card_y)
                card_w = min(iw - card_x, card_w)
                card_h = min(ih - card_y, card_h)

                # Dibujar el rectángulo de fondo con esquinas redondeadas
                corner_radius = int(0.1 * card_w)  # Ajuste del radio de las esquinas
                draw_rounded_rectangle(image_bgr, (card_x, card_y), (card_x + card_w, card_y + card_h), (255, 255, 255), corner_radius)

                # Superponer la región de la cara en la carta
                image_bgr[y:y+h, x:x+w] = face_region
                
                # Redimensionar el sombrero y calcular posición centrada
                new_hat_w = int(w * 1.3)  # Sombrero más ancho que la cara
                new_hat_h = int(hat_h * (new_hat_w / hat_w))  # Mantener proporción
                resized_hat = cv2.resize(hat, (new_hat_w, new_hat_h), interpolation=cv2.INTER_AREA)
                
                # Calcular la posición del sombrero centrado
                x_hat = x - int((new_hat_w - w) / 2)  # Centrar el sombrero horizontalmente
                y_hat = y - new_hat_h  # Posicionar el sombrero justo encima de la cara

                # Ajustar dimensiones para evitar que el sombrero se salga de los límites de la imagen
                hat_h_end = min(ih, y_hat + new_hat_h) - y_hat
                hat_w_end = min(iw, x_hat + new_hat_w) - x_hat
                resized_hat_cropped = resized_hat[:hat_h_end, :hat_w_end]

                # Superponer el sombrero con transparencia
                if y_hat >= 0:
                    for i in range(0, 3):  # BGR canales
                        image_bgr[y_hat:y_hat + hat_h_end, x_hat:x_hat + hat_w_end, i] = \
                            image_bgr[y_hat:y_hat + hat_h_end, x_hat:x_hat + hat_w_end, i] * (1 - resized_hat_cropped[:, :, 3] / 255.0) + \
                            resized_hat_cropped[:, :, i] * (resized_hat_cropped[:, :, 3] / 255.0)
                

                #Redimensionar el collar y calcular posición centrada
                new_ruff_w = int(w * 1.5)  # Collar más ancho que la cara
                new_ruff_h = int(ruff_h * (new_ruff_w / ruff_w))  # Mantener proporción
                resized_ruff = cv2.resize(ruff, (new_ruff_w, new_ruff_h), interpolation=cv2.INTER_AREA)

                # Calcular la posición del collar centrado
                x_ruff = x - int((new_ruff_w - w) / 2)  # Centrar el collar horizontalmente
                y_ruff = y + h - int(0.2 * new_ruff_h)  # Posicionar el collar justo debajo de la cara

                # Ajustar dimensiones para evitar que el collar se salga de los límites de la imagen
                ruff_h_end = min(ih, y_ruff + new_ruff_h) - y_ruff
                ruff_w_end = min(iw, x_ruff + new_ruff_w) - x_ruff
                resized_ruff_cropped = resized_ruff[:ruff_h_end, :ruff_w_end]

                # Superponer el collar con transparencia
                if y_ruff + ruff_h_end <= ih:
                    for i in range(0, 3):  # BGR canales
                        image_bgr[y_ruff:y_ruff + ruff_h_end, x_ruff:x_ruff + ruff_w_end, i] = \
                            image_bgr[y_ruff:y_ruff + ruff_h_end, x_ruff:x_ruff + ruff_w_end, i] * (1 - resized_ruff_cropped[:, :, 3] / 255.0) + \
                            resized_ruff_cropped[:, :, i] * (resized_ruff_cropped[:, :, 3] / 255.0)

        # Mostrar la imagen en una ventana
        cv2.imshow('Face Detection', image_bgr)

        # Presionar 'q' para salir
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()


In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Inicializar MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Inicializar la cámara
cap = cv2.VideoCapture(0)

# Índices de los landmarks que bordean la cara completamente (basado en el mapeo de MediaPipe)
face_outline_indices = [
    10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 
    378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 
    162, 21, 54, 103, 67, 109
]

# Función para dibujar un rectángulo con esquinas redondeadas
def draw_rounded_rectangle(image, top_left, bottom_right, color, corner_radius, thickness=-1):
    x1, y1 = top_left
    x2, y2 = bottom_right

    # Dibujar círculos en las esquinas
    cv2.circle(image, (x1 + corner_radius, y1 + corner_radius), corner_radius, color, thickness)
    cv2.circle(image, (x2 - corner_radius, y1 + corner_radius), corner_radius, color, thickness)
    cv2.circle(image, (x1 + corner_radius, y2 - corner_radius), corner_radius, color, thickness)
    cv2.circle(image, (x2 - corner_radius, y2 - corner_radius), corner_radius, color, thickness)

    # Dibujar líneas entre los círculos
    cv2.rectangle(image, (x1 + corner_radius, y1), (x2 - corner_radius, y2), color, thickness)
    cv2.rectangle(image, (x1, y1 + corner_radius), (x2, y2 - corner_radius), color, thickness)

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No se puede acceder a la cámara.")
            break

        # Convertir la imagen a RGB para procesarla con MediaPipe
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Procesar la imagen para detectar landmarks faciales
        results = face_mesh.process(image_rgb)
        
        # Convertir la imagen de nuevo a BGR para OpenCV
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # Verificar si se detectaron caras
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Obtener los puntos específicos que forman el contorno de la cara
                ih, iw, _ = image_bgr.shape
                contour_points = [
                    (int(face_landmarks.landmark[i].x * iw), int(face_landmarks.landmark[i].y * ih)) 
                    for i in face_outline_indices
                ]

                # Dibujar el contorno de la cara
                cv2.polylines(image_bgr, [np.array(contour_points)], isClosed=True, color=(255, 255, 255), thickness=2)

        # Mostrar el resultado
        cv2.imshow('Face Contour', image_bgr)

        # Salir con la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()


In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Inicializar MediaPipe Face Mesh (para obtener los landmarks)
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Cargar la imagen del sombrero y collar (sin cambios)
hat = cv2.imread('hat-crop.png', cv2.IMREAD_UNCHANGED)
hat_h, hat_w, _ = hat.shape

ruff = cv2.imread('ruff-crop.png', cv2.IMREAD_UNCHANGED)
ruff_h, ruff_w, _ = ruff.shape

# Inicializar la cámara
cap = cv2.VideoCapture(0)

# Índices de los landmarks que definen el contorno de la cara
face_outline_indices = [
    10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 
    378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 
    162, 21, 54, 103, 67, 109
]

# Función para dibujar un rectángulo con esquinas redondeadas
def draw_rounded_rectangle(image, top_left, bottom_right, color, corner_radius, thickness=-1):
    x1, y1 = top_left
    x2, y2 = bottom_right

    # Dibujar círculos en las esquinas
    cv2.circle(image, (x1 + corner_radius, y1 + corner_radius), corner_radius, color, thickness)
    cv2.circle(image, (x2 - corner_radius, y1 + corner_radius), corner_radius, color, thickness)
    cv2.circle(image, (x1 + corner_radius, y2 - corner_radius), corner_radius, color, thickness)
    cv2.circle(image, (x2 - corner_radius, y2 - corner_radius), corner_radius, color, thickness)

    # Dibujar líneas entre los círculos
    cv2.rectangle(image, (x1 + corner_radius, y1), (x2 - corner_radius, y2), color, thickness)
    cv2.rectangle(image, (x1, y1 + corner_radius), (x2, y2 - corner_radius), color, thickness)

# Inicializar Face Mesh
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No se puede acceder a la cámara.")
            break

        # Convertir la imagen a RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Procesar la imagen para obtener los landmarks de la cara
        results = face_mesh.process(image_rgb)

        # Convertir de nuevo a BGR para OpenCV
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                ih, iw, _ = image_bgr.shape
                
                # Obtener los puntos del contorno de la cara
                contour_points = [
                    (int(face_landmarks.landmark[i].x * iw), int(face_landmarks.landmark[i].y * ih)) 
                    for i in face_outline_indices
                ]
                
                # Crear una máscara para la cara (un área cerrada con los landmarks)
                mask = np.zeros((ih, iw), dtype=np.uint8)
                cv2.fillPoly(mask, [np.array(contour_points)], 255)

                # Crear una región de la cara usando la máscara
                face_region = cv2.bitwise_and(image_bgr, image_bgr, mask=mask)

                # Redibujar la carta
                # Calcular el tamaño de la carta basado en la cara detectada
                x, y, w, h = cv2.boundingRect(np.array(contour_points))  # Bounding box ajustada al contorno de la cara
                card_w = int(w * 1.4)
                card_h = int(card_w * 7 / 5)
                card_x = max(0, x - int((card_w - w) / 2))
                card_y = max(0, y - int(h * 0.3))

                # Limitar las dimensiones dentro de la imagen
                card_w = min(iw - card_x, card_w)
                card_h = min(ih - card_y, card_h)

                # Redibujar la carta (puedes añadir un fondo de color o transparencia aquí)
                corner_radius = int(0.1 * card_w)
                draw_rounded_rectangle(image_bgr, (card_x, card_y), (card_x + card_w, card_y + card_h), (255, 255, 255), corner_radius)

                
                # Redimensionar el sombrero y calcular posición centrada
                new_hat_w = int(w * 1.3)  # Sombrero más ancho que la cara
                new_hat_h = int(hat_h * (new_hat_w / hat_w))  # Mantener proporción
                resized_hat = cv2.resize(hat, (new_hat_w, new_hat_h), interpolation=cv2.INTER_AREA)
                
                # Calcular la posición del sombrero centrado
                x_hat = x - int((new_hat_w - w) / 2)  # Centrar el sombrero horizontalmente
                y_hat = y - new_hat_h  # Posicionar el sombrero justo encima de la cara

                # Ajustar dimensiones para evitar que el sombrero se salga de los límites de la imagen
                hat_h_end = min(ih, y_hat + new_hat_h) - y_hat
                hat_w_end = min(iw, x_hat + new_hat_w) - x_hat
                resized_hat_cropped = resized_hat[:hat_h_end, :hat_w_end]

                # Superponer el sombrero con transparencia
                if y_hat >= 0:
                    for i in range(0, 3):  # BGR canales
                        image_bgr[y_hat:y_hat + hat_h_end, x_hat:x_hat + hat_w_end, i] = \
                            image_bgr[y_hat:y_hat + hat_h_end, x_hat:x_hat + hat_w_end, i] * (1 - resized_hat_cropped[:, :, 3] / 255.0) + \
                            resized_hat_cropped[:, :, i] * (resized_hat_cropped[:, :, 3] / 255.0)
                

                #Redimensionar el collar y calcular posición centrada
                new_ruff_w = int(w * 1.5)  # Collar más ancho que la cara
                new_ruff_h = int(ruff_h * (new_ruff_w / ruff_w))  # Mantener proporción
                resized_ruff = cv2.resize(ruff, (new_ruff_w, new_ruff_h), interpolation=cv2.INTER_AREA)

                # Calcular la posición del collar centrado
                x_ruff = x - int((new_ruff_w - w) / 2)  # Centrar el collar horizontalmente
                y_ruff = y + h - int(0.2 * new_ruff_h)  # Posicionar el collar justo debajo de la cara

                # Ajustar dimensiones para evitar que el collar se salga de los límites de la imagen
                ruff_h_end = min(ih, y_ruff + new_ruff_h) - y_ruff
                ruff_w_end = min(iw, x_ruff + new_ruff_w) - x_ruff
                resized_ruff_cropped = resized_ruff[:ruff_h_end, :ruff_w_end]

                # Superponer el collar con transparencia
                if y_ruff + ruff_h_end <= ih:
                    for i in range(0, 3):  # BGR canales
                        image_bgr[y_ruff:y_ruff + ruff_h_end, x_ruff:x_ruff + ruff_w_end, i] = \
                            image_bgr[y_ruff:y_ruff + ruff_h_end, x_ruff:x_ruff + ruff_w_end, i] * (1 - resized_ruff_cropped[:, :, 3] / 255.0) + \
                            resized_ruff_cropped[:, :, i] * (resized_ruff_cropped[:, :, 3] / 255.0)


        # Mostrar la imagen con el filtro aplicado
        cv2.imshow('Face Filter', image_bgr)

        # Salir si se presiona la tecla 'q'
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()
